<a href="https://colab.research.google.com/github/jaehyun0220/Colab/blob/master/jigsaw_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 데이터 활용에 필요한 기본 패키지 로딩
import sys #access to system parameters 
print("Python version: {}". format(sys.version))

import pandas as pd
print("pandas version: {}". format(pd.__version__))

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import tensorflow as tf
print("tensorflow version: {}".format(tf.__version__))

import keras
print("keras version: {}".format(keras.__version__))

import os
#import io

Python version: 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
pandas version: 0.24.2
scikit-learn version: 0.21.1
NumPy version: 1.16.3
tensorflow version: 1.13.1
keras version: 2.2.4


Using TensorFlow backend.


In [0]:
#데이터 전처리 관련 라이브러리 로드
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import ShuffleSplit

# Deep Learning Model 로드
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation 
from keras.wrappers.scikit_learn import KerasClassifier

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#HyperParameter Tuning을 위한 라이브러리 로드
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#모델 평가를 위한 라이브러리 로드
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn import model_selection

#수학 & 통계 관련 라이브러리 로드
import scipy.stats as st
from collections import Counter

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Auth 인증 및 Google Drive 활용 Data load
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# Google Drive 내 Custom Class 경로 지정
import sys
sys.path.insert(0, '/gdrive/My Drive/CustomClasses')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
# from google.colab import files
# files.upload()
# !cp ./kaggle.json /root/.kaggle/kaggle.json
# !cp ./kaggle.json /gdrive/My\ Drive/CustomClasses/.kaggle/kaggle.json

In [0]:
# kaggle 실행을 위한 json file copy & 권한 부여
!cp /gdrive/My\ Drive/CustomClasses/.kaggle/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json
# !cat /gdrive/My\ Drive/CustomClasses/.kaggle/kaggle.json
# !kaggle competitions list

In [35]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -t /gdrive/My\ Drive/

  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 71.7MB/s]
 74% 9.00M/12.1M [00:00<00:00, 13.7MB/s]
100% 12.1M/12.1M [00:00<00:00, 13.1MB/s]
 97% 264M/273M [00:04<00:00, 75.6MB/s]
100% 273M/273M [00:04<00:00, 65.0MB/s]


In [39]:
!ls /content

adc.json     sample_data		test.csv.zip
kaggle.json  sample_submission.csv.zip	train.csv.zip


In [38]:
!cd /gdrive/My\ Drive/

'11월 cplj.pdf'		       CustomClasses		 SKeeperBackup
 버킷리스트_1808.xlsx	       MastersDataScience	 Study
'709동 1407호 입금 현황.xls'   nlp_kaggle_project.gdoc	 다변량통계.zip
'Colab Notebooks'	       Notability
